# 🌊 Extracting Wave Height from Sentinel-3 Altimetry Data

This notebook demonstrates how to extract **significant wave height** from Sentinel-3 SRAL altimetry data over the **Mediterranean Sea** using Python.

## 📥 Step 1: Load Sentinel-3 NetCDF Data

We use the `netCDF4` library to read the NetCDF file containing Sentinel-3 SRAL data.

## 🌍 Step 2: Filter Data for the Mediterranean Region

We define the bounding box for the Mediterranean Sea and filter the data accordingly.

## 📊 Step 3: Visualize Wave Height Distribution

We plot a histogram and a scatter map of wave height values over the Mediterranean Sea.

In [2]:
# Load required libraries
!pip install netCDF4
from netCDF4 import Dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 65.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 72.9 MB/s eta 0:00:00


In [8]:
from google.colab import drive
#drive.mount('/content/drive')


# OPTIONAL: cd into the folder where your notebook/file lives in Drive
%cd /content/drive/MyDrive/Colab Notebooks/Torino
!ls -lah  # sanity check: do you see your file here


# Replace with your file path
file_path = '/content/drive/MyDrive/Torino/standard_measurement.nc'

# Open the NetCDF file
nc = Dataset(file_path, mode='r')

# Print all variable names
print("Variables:")
print(nc.variables.keys())

# Print dimensions
print("\nDimensions:")
print(nc.dimensions.keys())

# Print global attributes
print("\nGlobal Attributes:")
print(nc.ncattrs())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks/Torino
total 1.6M
-rw------- 1 root root 1.7K Sep 26 11:33 '0 - First_steps.ipynb'
-rw------- 1 root root  18K Oct 23 09:32 '1 - Open_and_display.ipynb'
-rw------- 1 root root  16K Oct 23 09:52  2-Time_series.ipynb
-rw------- 1 root root 4.8K Oct 23 10:15  3-Algal_blooms.ipynb
-rw------- 1 root root 8.1K Oct 23 08:54  ee_api_colab_setup.ipynb
-rw------- 1 root root  11K Oct 23 10:22  Extract_Wave_Height_Sentinel3_Mediterranean.ipynb
-rw------- 1 root root 1.5M Oct  3 13:28  standard_measurement.nc


FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/Torino/standard_measurement.nc'

In [ ]:
# Print details of a specific variable
var_name = 'swh_ocean_01_ku'  # replace with one from your file
print(f"\nDetails of variable '{var_name}':")
print(nc.variables[var_name])

In [ ]:
# Print details of a specific variable
var_name = 'lat_01'  # replace with one from your file
print(f"\nDetails of variable '{var_name}':")
print(nc.variables[var_name])

In [ ]:
# Print details of a specific variable
var_name = 'lon_01'  # replace with one from your file
print(f"\nDetails of variable '{var_name}':")
print(nc.variables[var_name])

In [ ]:
# Extract variables
lat = nc.variables['lat_01'][:]
lon = nc.variables['lon_01'][:]
swh = nc.variables['swh_ocean_01_ku'][:]

# Close the NetCDF file
nc.close()

In [ ]:
plt.plot(lat, swh)
# Show the plot
plt.show()

In [ ]:
# Define bounding box
lat_min, lat_max = 35.0, 37.0

# Filter data within the region
mask = (lat >= lat_min) & (lat <= lat_max)
med_lat = lat[mask]
med_lon = lon[mask]
med_swh = swh[mask]

# Adjust lon values
med_lon_b = med_lon/1000

# Create a DataFrame
df = pd.DataFrame({'Latitude': med_lat, 'Longitude': med_lon_b, 'Wave_Height': med_swh})
df.head()

In [ ]:
# How many datapoints do I have?
df.shape

In [ ]:
df

In [ ]:
# Plot histogram of wave height
plt.figure(figsize=(10, 6))
plt.hist(df['Wave_Height'], bins=30, color='skyblue', edgecolor='black')
plt.title('Distribution of Significant Wave Height in the Mediterranean')
plt.xlabel('Wave Height (m)')
plt.ylabel('Frequency')
plt.grid(True)
plt.show()

In [ ]:
# Scatter plot of wave height over Mediterranean
plt.figure(figsize=(10, 6))
sc = plt.scatter(df['Longitude'], df['Latitude'], c=df['Wave_Height'], cmap='viridis', s=10)
plt.colorbar(sc, label='Wave Height (m)')
plt.title('Significant Wave Height over the Mediterranean Sea')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.grid(True)
plt.show()

In [ ]:
import folium

# Define the region of interest using latitude and longitude bounds
# Example: Mediterranean Sea region
min_lat, max_lat = 35.0, 37.0
min_lon, max_lon = 0, 0.5

# Calculate the center of the region
center_lat = (min_lat + max_lat) / 2
center_lon = (min_lon + max_lon) / 2

# Create a folium map centered on the region of interest
m = folium.Map(location=[center_lat, center_lon], zoom_start=5)


# Add circle markers for each point in the DataFrame
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['Latitude'], row['Longitude']],
        radius=5,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.6,
        popup=f"Wave Height: {row['Wave_Height']} m"
    ).add_to(m)
m
